In [1]:
# 환경설정
from dotenv import load_dotenv

load_dotenv()

True

### chain을 이용한 Simple LLm
1. PromptTemplate
2. LLM
3. OutputParser

In [2]:
# 1. PromptTemplate
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate(
    template="{question}?",
    input_variables=["question"]
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ('system', '너는 애니메이션을 잘 아는 챗봇이야. 사용자의 질문에 친절하고 상세하게 답변 할 수 있어.'),
    ('user', '{question}')
])

In [4]:
# 2. Model
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0
)

In [5]:
# 3. OutputParser(StrOutputParser)
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [6]:
# 4. Chain -> 질의
# questiom: 어떤 포켓몬이 가장 강해?
chain= prompt | model | output_parser

question = "어떤 포켓몬이 가장 강해?"
response = chain.invoke({"question": question})

print(response)

포켓몬의 강함은 여러 요소에 따라 달라질 수 있습니다. 일반적으로는 전투 능력, 타입 상성, 기술 조합, 그리고 전략적인 사용 등이 중요한 요소입니다. 

현재 메타에서 강력한 포켓몬으로는 "아르세우스", "메가 레쿠쟈", "무장조" 등이 자주 언급됩니다. 하지만 특정 상황이나 대전 방식에 따라 강한 포켓몬이 달라질 수 있으니, 어떤 포켓몬이 가장 강한지는 상황에 따라 다를 수 있습니다. 

또한, 각 세대마다 새로운 포켓몬이 추가되기 때문에, 최신 정보를 참고하는 것이 좋습니다. 어떤 포켓몬을 사용하고 싶은지에 따라 전략을 세우는 것이 중요합니다!


In [7]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(prompt, model, output_parser)
chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

'포켓몬의 강함은 여러 요소에 따라 달라질 수 있습니다. 일반적으로는 전투 능력, 타입 상성, 기술 조합, 그리고 전략적인 사용 등이 중요한 요소입니다. \n\n현재 메타에서 강력한 포켓몬으로는 "아르세우스", "메가 레쿠쟈", "무장조", "이브이"의 진화형인 "리피아"와 "블래키" 등이 있습니다. 하지만 특정 상황이나 대전 방식에 따라 강한 포켓몬이 달라질 수 있으니, 어떤 포켓몬이 가장 강한지는 상황에 따라 다를 수 있습니다. \n\n또한, 포켓몬 게임의 업데이트나 새로운 세대가 출시되면 강력한 포켓몬이 바뀔 수 있으니, 최신 정보를 참고하는 것이 좋습니다!'

### 단계별 Chatbot
- 첫 대화에서 내 이름을 알려주고, 다음 대화에서 내 이름 기억하는지 물어보기!

1. 그냥 Chat
- langchain_openai의 ChatopenAI
- langchain_core.messages의 클래스

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.5
)

In [11]:
from langchain_core.messages import HumanMessage

model.invoke([
    HumanMessage('안녕! 내 이름은 주노야')
]).content

'안녕, 주노! 만나서 반가워. 어떻게 지내?'

In [12]:
model.invoke([
    HumanMessage('내 이름이 뭐라고 했지?')
]).content

'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 말씀해 주시면 기억하겠습니다!'

2. 직접 대화 맥락 유지
- langchain_opai의 ChatOpenAI
- langchain_core.messages의 클래스

In [14]:
from langchain_core.messages import HumanMessage, AIMessage

context = []

context.append(HumanMessage(content="안녕! 내 이름은 주노야"))

ai_response = model.invoke(context)
context.append(AIMessage(content=ai_response.content))

for message in context:
    print(f"{message.type}: {message.content}")

context.append(HumanMessage(content="내 이름이 뭐라고 했지?"))

ai_response = model.invoke(context)
context.append(AIMessage(content=ai_response.content))

for message in context:
    print(f"{message.type}: {message.content}")

human: 안녕! 내 이름은 주노야
ai: 안녕하세요, 주노! 만나서 반가워요. 어떻게 도와드릴까요?
human: 안녕! 내 이름은 주노야
ai: 안녕하세요, 주노! 만나서 반가워요. 어떻게 도와드릴까요?
human: 내 이름이 뭐라고 했지?
ai: 당신의 이름은 주노라고 하셨습니다! 맞나요?


3. Memory로 대화 맥락 유지 
- langchain_opai의 ChatOpenAI
- langchain_core.messages의 클래스
- langchain_core.runnables의 클래스
- langchain_core.prompts의 클래스

In [38]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []
    
    def add_messages(self, messages):
        self.messages.extend(messages)
    
    def clear(self):
        self.messages = []
        
    def __repr__(self):
        return str(self.messages)


prompt = ChatPromptTemplate.from_messages([
    ('system', '{skill}을 잘하는 AI 어시스턴트야'), 
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])

store = {}  

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model


chat_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

response = chat_with_history.invoke(
    {'skill': '대화', 'query': '인녕! 내 이름은 주노야.'},
    config={'configurable': {'session_id': '주노'}}
)

print(response.content)

response = chat_with_history.invoke(
    {'skill': '대화', 'query':'내 이름이 뭐라고?'},
    config={'configurable':{'session_id':'주노'}}
)

print(response.content)

안녕, 주노! 만나서 반가워. 어떻게 지내고 있어?
주노라고 했지! 맞지?
